In [1]:
%reload_ext autoreload
%autoreload 1

In [2]:
import numpy as np
from time import time

In [3]:
import cProfile
import pstats

In [4]:
%aimport malariagen_data.ag3

In [5]:
%aimport malariagen_data.veff 

In [6]:
veff = malariagen_data.veff

In [15]:
ag3 = malariagen_data.Ag3("simplecache::gs://vo_agam_release", simplecache=dict(cache_storage="gcs_cache"))

In [16]:
genome = ag3.open_genome()
genome

<zarr.hierarchy.Group '/'>

In [17]:
list(genome)

['2L', '2R', '3L', '3R', 'Mt', 'UNKN', 'X', 'Y_unplaced']

In [18]:
genome["3R"][0:10].tobytes().decode()

'CCTCTACGTT'

In [20]:
geneset = ag3.geneset()
geneset

,seqid,source,type,start,end,score,strand,phase,ID,Parent,Name
0,2L,VectorBase,chromosome,1,49364325,NaN,NaN,NaN,2L,NaN,NaN
1,2L,VectorBase,gene,157348,186936,NaN,-,NaN,AGAP004677,NaN,NaN
2,2L,VectorBase,mRNA,157348,181305,NaN,-,NaN,AGAP004677-RA,AGAP004677,NaN
3,2L,VectorBase,three_prime_UTR,157348,157495,NaN,-,NaN,NaN,AGAP004677-RA,NaN
4,2L,VectorBase,exon,157348,157623,NaN,-,NaN,NaN,AGAP004677-RA,AGAP004677-RB-E4
...,...,...,...,...,...,...,...,...,...,...,...
196140,Y_unplaced,VectorBase,five_prime_UTR,47932,48111,NaN,+,NaN,NaN,AGAP029375-RA,NaN
196141,Y_unplaced,VectorBase,exon,47932,48138,NaN,+,NaN,NaN,AGAP029375-RA,AGAP029375-RA-E2
196142,Y_unplaced,VectorBase,CDS,48112,48138,NaN,+,0.0,AGAP029375-PA,AGAP029375-RA,NaN
196143,Y_unplaced,VectorBase,exon,48301,48385,NaN,+,NaN,NaN,AGAP029375-RA,AGAP029375-RA-E3


In [21]:
%%time
# this will take some time, loading gff, one-off cost
ann = veff.Annotator(
    genome=genome,
    geneset=geneset,
)

CPU times: user 18.9 s, sys: 51.5 ms, total: 19 s
Wall time: 19 s


In [23]:
%%time
for effect in ann.get_effects(chrom='2L', pos=2422652, ref='A', alt='T',
                                  transcript_ids=["AGAP004707-RD"]):
       print(effect)

VariantEffect(effect='NON_SYNONYMOUS_CODING', impact='MODERATE', chrom='2L', pos=2422652, ref='A', alt='T', vlen=0, ref_start=2422652, ref_stop=2422652, gene_id='AGAP004707', gene_start=2358158, gene_stop=2431617, gene_strand='+', transcript_id='AGAP004707-RD', transcript_start=2358158, transcript_stop=2431617, transcript_strand='+', cds_id='AGAP004707-PD', cds_start=2422468, cds_stop=2422655, cds_strand='+', intron_start=None, intron_stop=None, intron_5prime_dist=None, intron_3prime_dist=None, intron_cds_5prime=None, intron_cds_3prime=None, ref_cds_start=2984, ref_cds_stop=2984, ref_intron_start=None, ref_intron_stop=None, ref_start_phase=2, ref_codon='ttA', alt_codon='ttT', codon_change='ttA/ttT', aa_pos=995, ref_aa='L', alt_aa='F', aa_change='L995F')
CPU times: user 0 ns, sys: 1.15 ms, total: 1.15 ms
Wall time: 1.09 ms


In [24]:
for a in effect:
    print(a)

NON_SYNONYMOUS_CODING
MODERATE
2L
2422652
A
T
0
2422652
2422652
AGAP004707
2358158
2431617
+
AGAP004707-RD
2358158
2431617
+
AGAP004707-PD
2422468
2422655
+
None
None
None
None
None
None
2984
2984
None
None
2
ttA
ttT
ttA/ttT
995
L
F
L995F


In [25]:
effect.effect, effect.impact

('NON_SYNONYMOUS_CODING', 'MODERATE')

In [26]:
effect.ref_codon, effect.alt_codon

('ttA', 'ttT')

In [27]:
effect.aa_pos, effect.ref_aa, effect.alt_aa, effect.aa_change

(995, 'L', 'F', 'L995F')

In [28]:
type(effect.aa_pos)

int

In [29]:
pos, ref, alt = ag3.snp_sites('2L')

In [30]:
import allel

In [31]:
start_idx = allel.SortedIndex(pos).locate_key(2429745)
start_idx

2150929

In [32]:
pp = pos[start_idx:start_idx+100].compute()
rr = ref[start_idx:start_idx+100].compute()
aaa = alt[start_idx:start_idx+100].compute()

In [33]:
def testf(n, show=False):
    # loop over sites
    for i, (p, r, aa) in enumerate(zip(pp, rr, aaa)):
        if i < n:
            # loop over alt alleles
            for a in aa:
                for effect in ann.get_effects(chrom='2L', pos=p, ref=r.decode(), alt=a.decode(),
                                              transcript_ids=["AGAP004707-RA"]):
                    if show:
                        print(effect)
    

In [34]:
%%time
testf(20)

CPU times: user 50.7 ms, sys: 88 µs, total: 50.8 ms
Wall time: 49.7 ms


In [35]:
%%time
x = ref.compute()

CPU times: user 1.23 s, sys: 168 ms, total: 1.4 s
Wall time: 4.12 s


In [36]:
x.nbytes

48525747

In [37]:
cProfile.run('testf(100)', sort='cumtime')

         468523 function calls (464545 primitive calls) in 0.239 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.239    0.239 {built-in method builtins.exec}
        1    0.000    0.000    0.239    0.239 <string>:1(<module>)
        1    0.001    0.001    0.239    0.239 <ipython-input-33-ac85f549eef5>:1(testf)
      600    0.001    0.000    0.238    0.000 veff.py:134(get_effects)
      600    0.002    0.000    0.139    0.000 veff.py:219(_get_gene_effects)
      600    0.005    0.000    0.128    0.000 veff.py:257(_get_transcript_effects)
      600    0.002    0.000    0.120    0.000 veff.py:324(_get_within_transcript_effects)
      300    0.000    0.000    0.085    0.000 veff.py:57(find)
      300    0.000    0.000    0.085    0.000 intervals.py:216(search)
      300    0.001    0.000    0.084    0.000 intervals.py:190(_search_tree)
      300    0.001    0.000    0.062    0.000 intervaltree

In [38]:
cProfile.run('testf(20)', sort='time')

         118564 function calls (117784 primitive calls) in 0.063 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    22620    0.010    0.000    0.012    0.000 base.py:570(__getattr__)
    25020    0.007    0.000    0.007    0.000 interval.py:159(__eq__)
   840/60    0.005    0.000    0.017    0.000 node.py:309(search_point)
     2460    0.004    0.000    0.012    0.000 {method 'add' of 'set' objects}
       60    0.004    0.000    0.006    0.000 veff.py:355(<listcomp>)
     9960    0.003    0.000    0.003    0.000 interval.py:173(__cmp__)
    24480    0.003    0.000    0.003    0.000 {method 'index' of 'list' objects}
      180    0.002    0.000    0.010    0.000 {built-in method builtins.sorted}
       60    0.002    0.000    0.002    0.000 veff.py:359(<listcomp>)
     9960    0.002    0.000    0.005    0.000 interval.py:204(__lt__)
       60    0.001    0.000    0.003    0.000 veff.py:333(<listcomp>)
       60    0.001   

In [39]:
ann.get_feature("AGAP004707-RD")

('2L',
 'VectorBase',
 'mRNA',
 2358158,
 2431617,
 nan,
 '+',
 nan,
 'AGAP004707-RD',
 'AGAP004707',
 nan)

In [40]:
ann.get_children("AGAP004707")

[('2L',
  'VectorBase',
  'mRNA',
  2358158,
  2431617,
  nan,
  '+',
  nan,
  'AGAP004707-RA',
  'AGAP004707',
  nan),
 ('2L',
  'VectorBase',
  'mRNA',
  2358158,
  2431617,
  nan,
  '+',
  nan,
  'AGAP004707-RB',
  'AGAP004707',
  nan),
 ('2L',
  'VectorBase',
  'mRNA',
  2358158,
  2431617,
  nan,
  '+',
  nan,
  'AGAP004707-RC',
  'AGAP004707',
  nan),
 ('2L',
  'VectorBase',
  'mRNA',
  2358158,
  2431617,
  nan,
  '+',
  nan,
  'AGAP004707-RD',
  'AGAP004707',
  nan),
 ('2L',
  'VectorBase',
  'mRNA',
  2358158,
  2431617,
  nan,
  '+',
  nan,
  'AGAP004707-RE',
  'AGAP004707',
  nan),
 ('2L',
  'VectorBase',
  'mRNA',
  2358158,
  2431617,
  nan,
  '+',
  nan,
  'AGAP004707-RF',
  'AGAP004707',
  nan),
 ('2L',
  'VectorBase',
  'mRNA',
  2358158,
  2431617,
  nan,
  '+',
  nan,
  'AGAP004707-RG',
  'AGAP004707',
  nan),
 ('2L',
  'VectorBase',
  'mRNA',
  2358158,
  2431617,
  nan,
  '+',
  nan,
  'AGAP004707-RH',
  'AGAP004707',
  nan),
 ('2L',
  'VectorBase',
  'mRNA',
  2358

In [41]:
ann.find("2L", 2358158, 2358159)

[('2L',
  'VectorBase',
  'chromosome',
  1,
  49364325,
  nan,
  nan,
  nan,
  '2L',
  nan,
  nan),
 ('2L',
  'VectorBase',
  'CDS',
  2358158,
  2358304,
  nan,
  '+',
  0.0,
  'AGAP004707-PE',
  'AGAP004707-RE',
  nan),
 ('2L',
  'VectorBase',
  'CDS',
  2358158,
  2358304,
  nan,
  '+',
  0.0,
  'AGAP004707-PK',
  'AGAP004707-RK',
  nan),
 ('2L',
  'VectorBase',
  'CDS',
  2358158,
  2358304,
  nan,
  '+',
  0.0,
  'AGAP004707-PI',
  'AGAP004707-RI',
  nan),
 ('2L',
  'VectorBase',
  'CDS',
  2358158,
  2358304,
  nan,
  '+',
  0.0,
  'AGAP004707-PA',
  'AGAP004707-RA',
  nan),
 ('2L',
  'VectorBase',
  'CDS',
  2358158,
  2358304,
  nan,
  '+',
  0.0,
  'AGAP004707-PB',
  'AGAP004707-RB',
  nan),
 ('2L',
  'VectorBase',
  'CDS',
  2358158,
  2358304,
  nan,
  '+',
  0.0,
  'AGAP004707-PD',
  'AGAP004707-RD',
  nan),
 ('2L',
  'VectorBase',
  'CDS',
  2358158,
  2358304,
  nan,
  '+',
  0.0,
  'AGAP004707-PJ',
  'AGAP004707-RJ',
  nan),
 ('2L',
  'VectorBase',
  'CDS',
  2358158,
 

In [44]:
%%time
df = ag3.snp_effects('AGAP004707-RA', site_mask='gamb_colu')

transcript : AGAP004707-RA
chromosome : 2L 
start : 2358158
stop : 2431617
strand : +
CPU times: user 46.8 s, sys: 486 ms, total: 47.3 s
Wall time: 46.9 s


In [45]:
df

,position,ref_allele,alt_alleles,effect,impact,ref_codon,alt_codon,aa_pos,ref_aa,alt_aa,aa_change
0,2358158,A,C,START_LOST,HIGH,Atg,Ctg,1.0,M,L,M1L
1,2358158,A,T,START_LOST,HIGH,Atg,Ttg,1.0,M,L,M1L
2,2358158,A,G,START_LOST,HIGH,Atg,Gtg,1.0,M,V,M1V
3,2358159,T,A,NON_SYNONYMOUS_CODING,MODERATE,aTg,aAg,1.0,M,K,M1K
4,2358159,T,C,NON_SYNONYMOUS_CODING,MODERATE,aTg,aCg,1.0,M,T,M1T
...,...,...,...,...,...,...,...,...,...,...,...
132301,2431616,G,C,STOP_LOST,HIGH,tGa,tCa,2065.0,*,S,*2065S
132302,2431616,G,T,STOP_LOST,HIGH,tGa,tTa,2065.0,*,L,*2065L
132303,2431617,A,C,STOP_LOST,HIGH,tgA,tgC,2065.0,*,C,*2065C
132304,2431617,A,T,STOP_LOST,HIGH,tgA,tgT,2065.0,*,C,*2065C


# SNP allele frequencies

In [131]:
%%time
df_meta, gt = ag3.snp_allele_frequencies("AGAP004707-RD", "gamb_colu")

transcript : AGAP004707-RD
chromosome : 2L 
start : 2358158
stop : 2431617
strand : +


/home/chris/.cache/pypoetry/virtualenvs/malariagen-data-mfr76-fD-py3.6/lib/python3.6/site-packages/dask/array/routines.py:1247: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  a = a[tuple(condition if i == axis else slice(None) for i in range(a.ndim))]


CPU times: user 44.4 s, sys: 7.24 s, total: 51.7 s
Wall time: 3min 10s


In [138]:
len(gt) *3

132306

In [123]:
df_meta.loc[df_meta.country == 'AG1000G-UG']


,sample_id,partner_sample_id,contributor,country,location,year,month,latitude,longitude,sex_call,sample_set,release,aim_fraction_colu,aim_fraction_arab,species_gambcolu_arabiensis,species_gambiae_coluzzii,species
2494,AC0007-C,1_A2,Martin Donnelly,Uganda,Nagongera,2012,10,0.770,34.026,F,AG1000G-UG,v3,0.458,0.741,arabiensis,NaN,arabiensis
2495,AC0008-C,1_A6,Martin Donnelly,Uganda,Nagongera,2012,10,0.770,34.026,F,AG1000G-UG,v3,0.456,0.740,arabiensis,NaN,arabiensis
2496,AC0009-Cx,1_A7,Martin Donnelly,Uganda,Nagongera,2012,10,0.770,34.026,F,AG1000G-UG,v3,0.458,0.746,arabiensis,NaN,arabiensis
2497,AC0010-C,1_A9,Martin Donnelly,Uganda,Nagongera,2012,10,0.770,34.026,F,AG1000G-UG,v3,0.452,0.743,arabiensis,NaN,arabiensis
2498,AC0011-C,1_B5,Martin Donnelly,Uganda,Nagongera,2012,10,0.770,34.026,F,AG1000G-UG,v3,0.455,0.745,arabiensis,NaN,arabiensis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2779,AC0295-C,K92,Martin Donnelly,Uganda,Kihihi,2012,11,-0.751,29.701,F,AG1000G-UG,v3,0.026,0.002,gamb_colu,gambiae,gambiae
2780,AC0296-C,K93,Martin Donnelly,Uganda,Kihihi,2012,11,-0.751,29.701,F,AG1000G-UG,v3,0.029,0.003,gamb_colu,gambiae,gambiae
2781,AC0297-C,K94,Martin Donnelly,Uganda,Kihihi,2012,11,-0.751,29.701,F,AG1000G-UG,v3,0.026,0.002,gamb_colu,gambiae,gambiae
2782,AC0298-C,K95,Martin Donnelly,Uganda,Kihihi,2012,11,-0.751,29.701,F,AG1000G-UG,v3,0.029,0.002,gamb_colu,gambiae,gambiae
